In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

datadir = Path('/work/data/mushtaq/Plate2_output')
#datadir = Path('/home/hajaalin/tmp/toiba/output')

# create folder for  output
outputdir = datadir / '20200921b'
outputdir.mkdir( exist_ok=True)

# create folder for debugging output
debugdir = outputdir / 'debug'
debugdir.mkdir( exist_ok=True)

wells = datadir.glob('Well*')
wells = sorted(wells)

# this will contain all data
df = pd.DataFrame()

# exclude images before this timepoint (can be used to exclude restarts in the beginning of the dataset)
exclude_before = pd.to_datetime('2020_09_10_15_58', format='%Y_%m_%d_%H_%M')

# list of dataframes, one per .csv
tmps = []

for w in wells:
    print(w)
    files = (w / "ar").glob("*.csv")
    files = sorted(files)
    
    # extra index for cycles (can be useful unless imaging is stopped and restarted)
    cycle = 1
    
    
    for f in files:
        #print(f)
        
        # read only AR data
        tmp = pd.read_csv(f, usecols=['AR'])
        
        # add a column that shows which well the numbers are from
        tmp["Well_Site"] = w.stem
        tmp[["Well","Site"]] = tmp["Well_Site"].str.extract('(?P<Well>.*)(?P<Site>_[0-9]{1,2}$)', expand=True)
        
        # extract timepoint
        timestr = f.stem.replace("_segmentation_results","")
        timestr = timestr.split('_')
        timestr = "_".join(timestr[:5])
        timepoint = pd.to_datetime(timestr, format='%Y_%m_%d_%H_%M')
        
        if timepoint < exclude_before:
            continue
        
        # add column for timepoint
        tmp['Timepoint'] = timepoint
        # add column for cycle
        tmp['Cycle'] = cycle
        
        # add to the list of dataframes
        tmps.append(tmp)
        
        cycle = cycle + 1
        #break
    #break

# combine per .csv dataframes
df = pd.concat(tmps, ignore_index=True)
    
print('Done.')



In [ ]:
df.tail()

In [ ]:
# add a column that show time from start of experiment
start_time = df['Timepoint'].min()
df['T'] = df['Timepoint'] - start_time

df.head()

In [ ]:
# keep a copy of original to avoid re-reading data from disk in case there is a problem in the fixes below
df0 = df.copy()

In [ ]:
# Calculate mean AR for each combination of well, cycle and timepoint.
# The aim is to find cases where one cycle contains multiple timepoints.
wct = df[['Well','Cycle','T','AR']].groupby(['Well','Cycle','T']).mean()
wct.reset_index(inplace=True)

# save the data for checking in excel
wct.to_csv(debugdir / 'wct.csv')

# Find cases where there are multiple entries for the same cycle, 
# i.e. when all fields were not imaged within the same minute.

#             keep
# value False first last    False^last
# A       0     0     0          0
# B       1     0     1          0
# B       1     1     1          0
# B       1     1     0          1
# C       0     0     0          0
duplicated_all = wct.duplicated(subset=['Well','Cycle'], keep=False)
duplicated_keep_last = wct.duplicated(subset=['Well','Cycle'], keep='last')
duplicated_last = duplicated_all ^ duplicated_keep_last

wct[duplicated_all].to_csv(debugdir /  'duplicated_all.csv')
wct[duplicated_last].to_csv(debugdir / 'duplicated_last.csv')

# consider that the last of the duplicated lines has the correct timestamp
fix = wct[duplicated_last]

# images from same well within this timedelta are considered to belong to the same timepoint
max_cycle_timedelta_within_well = np.timedelta64(4,'m')

for i in fix.index:
    w = fix.loc[i]['Well']
    c = fix.loc[i]['Cycle']
    t = fix.loc[i]['T']
    #print( w + ' ' + str(c) + ' ' + str(t))
    well_cycle = df[(df['Well'] == w) & (df['Cycle']==c)]
    idx = well_cycle.index
    
    # check how many timepoints there are for this cycle, alert if > 2
    if well_cycle['Timepoint'].unique().size > 2:
        print('WARNING: more than 2 timepoints per cycle: ' + str(w) + ', cycle ' + str(c))
        well_cycle.to_csv(debugdir / (str(w) + '_cycle' + str(c) + '.csv'))
    
    # check how far apart the timepoints are, alert if > limit
    well_cycle_duration = well_cycle['Timepoint'].max() - well_cycle['Timepoint'].min()
    if well_cycle_duration > max_cycle_timedelta_within_well:
        print('WARNING: too long well cycle time: ' + str(w) + ', cycle ' + str(c))
        well_cycle.to_csv(debugdir / (str(w) + '_cycle' + str(c) + '.csv'))
    
    # set the same T to all entries in this well/cycle combination
    df.at[idx, 'T'] = t


In [ ]:
# write data to one .csv
df0.to_csv(debugdir / 'all_measurements_in_one_file_before_fixes.csv', index=False)

# write data to one .csv
df.to_csv(outputdir / 'all_measurements_in_one_file_after_fixes.csv', index=False)


In [ ]:
df.info()

In [ ]:
# convert timedelta to hours
df['h'] = df['T'] / pd.Timedelta(1, 'h')

# combine the AR data in another dataframe, take mean over timepoint
mean_ar = pd.pivot_table(df, index='h', columns='Well', values='AR', aggfunc='mean')

# After the above pivot operation, T is the index of the dataframe.
# reset_index() will make T a normal column.

# write mean AR data in one .csv
mean_ar.reset_index().to_csv(outputdir / 'mean_ar_all_in_one_file.csv', index=False)

mean_ar.reset_index().head()

In [ ]:
mean_ar.head()

In [ ]:
import matplotlib.pyplot as plt

# prepare to plot data
fig,ax = plt.subplots(figsize=(25,15))

# loop over wells
for c in mean_ar.columns:
    #print(c)
    
    # The NaN values on mean_ar cause problems for plotting.
    # Workaround is to... create a new dataframe, keep
    keep = [c]
    # ...make copy of the data
    tmp = mean_ar.copy()
    # ...keep only the interesting column (well)
    tmp = tmp.drop(tmp.columns.difference(keep), axis=1) 
    # ...drop NaN values
    tmp = tmp.dropna()

    # plot the data
    tmp.plot(ax=ax)



# y-axis range for figure
MEAN_AR_MIN = 1.3
MEAN_AR_MAX = 2
plt.ylim(MEAN_AR_MIN, MEAN_AR_MAX)

# x-axis range
TIME_MIN = 0
TIME_MAX = 26

ax.set_xlim(TIME_MIN, TIME_MAX)

# set y axis label
plt.ylabel('Mean AR')

plt.savefig(outputdir / 'mean_ar.png')

#tmp.head()

In [ ]:
tmp.head()

In [ ]:
tmp.info()